In [1]:
import pandas as pd

In het bestand "DE Sales.csv" zijn een aantal zaken corrupt:

* Regel 96209-96214 hebben een puntkomma ; in plaats van een komma , als scheidingsteken
* Regel 98817-98830 heeft een Nederlands datumformaat in plaats van een Amerikaans (28-7-2004 ipv 7/28/2004)
* Regel 179482-179484 zijn "echt" corrupt
* Regel 232710 t/m 232719 hebben geen laatste kolom


In [20]:
df = pd.read_csv("DE Sales.csv")

Inlezen lijkt goed te gaan..

In [21]:
df.head()

,ProductID,Date,Zip,Units,Revenue,Country
0,725,1/15/1999,41540.0,1.0,115.45,Germany
1,787,6/6/2002,41540.0,1.0,314.95,Germany
2,788,6/6/2002,41540.0,1.0,314.95,Germany
3,940,1/15/1999,22587.0,1.0,687.70,Germany
4,396,1/15/1999,22587.0,1.0,857.06,Germany


Maar vermoedelijk komt dat doordat het datatype van "date" niet herkend is:

In [22]:
# Get the type of each column
df.dtypes

ProductID     object
Date          object
Zip          float64
Units        float64
Revenue      float64
Country       object
dtype: object

In [23]:
# Laten we de datum "echt" als datum proberen te parsen:
df['Date'] = pd.to_datetime(df['Date'], format="mixed")



DateParseError: Unknown datetime string format, unable to parse: 893qgunheja9;gpuivajnr, at position 3155

De "echte" vervuiling speelt nu op. Optie 1 is simpelweg het weglaten van fouten:

In [25]:
# Replace values that do not convert with "NaT"
df = pd.to_datetime(df['Date'], format="mixed", errors="coerce")


Dat blijkt niet zo'n probleem voor Pandas - de "mixed" kan verschillende datum-types herkennen. (Kan gevaarlijk zijn, maar in de "gevaarlijke" situaties zou je afwijkingen ook niet herkennen!)

Voor de eerste set niet zo'n probleem (door een verkeerd scheidingsteken staat alles in de eerste kolom:) 

In [19]:
# Select rows 96209 until 96214
df[96209:96215]


,ProductID,Date,Zip,Units,Revenue,Country
96209,631;7/27/2004;20251;1;398.74;Germany,NaT,NaN,NaN,NaN,NaN
96210,645;7/27/2004;14165;1;734.74;Germany,NaT,NaN,NaN,NaN,NaN
96211,791;7/27/2004;12627;1;62.95;Germany,NaT,NaN,NaN,NaN,NaN
96212,792;7/27/2004;13465;1;62.95;Germany,NaT,NaN,NaN,NaN,NaN
96213,572,2004-07-29,10783.0,1.0,445.99,Germany
96214,791,2004-07-29,13597.0,1.0,62.95,Germany


Voor de tweede fout:

In [18]:
df[98817:98830]

,ProductID,Date,Zip,Units,Revenue,Country
98817,992,2004-07-28,40229.0,1.0,278.20,Germany
98818,1000,2004-07-28,22111.0,1.0,91.82,Germany
98819,2054,2004-07-28,40215.0,1.0,551.20,Germany
98820,396,2004-07-28,41516.0,1.0,907.99,Germany
98821,635,2004-07-28,22419.0,1.0,884.36,Germany
98822,635,2004-07-28,40229.0,1.0,884.36,Germany
98823,635,2004-07-28,42107.0,1.0,884.36,Germany
98824,689,2004-07-28,41199.0,1.0,340.99,Germany
98825,1168,2004-07-28,22393.0,1.0,367.45,Germany
98826,2222,2004-07-28,41169.0,1.0,225.70,Germany
